In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

all_words = []
for txt in corpus:
    words = word_tokenize(txt)
    for w in words:
        if w not in stop_words:
            all_words.append(w)

all_words = nltk.FreqDist(all_words)

In [ ]:
processed = data['headline'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',' ')
processed = processed.str.replace(r'[^\w\d\s]', ' ')
processed = processed.str.replace(r'\s+', ' ')
processed = processed.str.replace(r'^\s+|\s+?$', ' ')
processed = processed.str.replace(r'\d+',' ')
processed = processed.str.lower()

stop_words = set(stopwords.words('english'))
processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

all_words = []
for txt in processed:
    words = word_tokenize(txt)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

In [ ]:
word_features = list(all_words.keys())[:5000]

def find_features(headings):
    words = word_tokenize(headings)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

headings = list(zip(processed, data['is_sarcastic']))

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn import model_selection

featuresets = [(find_features(text), label) for (text, label) in headings]

In [ ]:
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=1)
model = MultinomialNB()
nl_model = SklearnClassifier(model)
nl_model.train(training)
accuracy = nltk.classify.accuracy(nl_model, testing)*100
print("Naive Bayes Accuracy: {}".format(accuracy))

txt_features, labels = list(zip(*testing))
prediction = nl_model.classify_many(txt_features)



In [ ]:
sarc = [];
no_sarc = [];
unifyy = list(zip(prediction, txt_features, labels))
for prd, txt, lbl in unifyy:
    for key, value in txt.items():
        if value == True and lbl == prd == 0:
            no_sarc.append(key)
            break
        elif value == True and lbl == prd == 1:
            sarc.append(key)
            break

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pylab as pl

sarc_words = plt.figure()
pl.figure(figsize = (10,10))
sarcastic = str(sarc)
wordCloud = WordCloud(background_color = "black").generate(sarcastic)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Sarcasm Words', fontsize = 13)
plt.show()
sarc_words.savefig("Sarcasm Words.jpg")

In [ ]:
non_sarc = plt.figure()
pl.figure(figsize = (10,10))
not_sarcastic = str(no_sarc)
wordCloud = WordCloud(background_color = "black").generate(not_sarcastic)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Not Sarcasm Words', fontsize = 13)
plt.show()
non_sarc.savefig("Non sarcasm words.jpg")